In [5]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Input, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
import cv2
import numpy as np
import os
import shutil
import zipfile
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import requests

In [6]:


# Define a new input layer that accepts grayscale (1-channel) images
input_layer = Input(shape=(224, 224, 1))

# Use Conv2D to change 1-channel input to 3-channel so it can work with MobileNetV2
x = Conv2D(3, (3, 3), padding='same', activation='relu')(input_layer)

# Load pre-trained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Modify the first layer to accept 1-channel input and broadcast to 3 channels
input_layer = Input(shape=(224, 224, 1))  # Grayscale input
x = Lambda(lambda x: tf.image.grayscale_to_rgb(x))(input_layer)  # Convert grayscale to RGB

# Connect the base model
base_model_output = base_model(x)

# Add custom layers on top of the base model
x = Dropout(0.5)(x)  # Added Dropout for regularization
x = Dense(64, activation='relu')(x)  # Added Dense layer for more complexity

x = GlobalAveragePooling2D()(base_model_output)
x = Dense(1, activation='sigmoid')(x)  # For binary classification (person vs non-person)

# Create the final model
model = Model(inputs=input_layer, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
# model = tf.keras.models.load_model('model/model_geeks.h5')
model.load_weights('model/train_3.keras')

/Users/nabil/Documents/Github/FYP/myenv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 114 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [9]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_2 (Lambda)               │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 2,225,153 (8.49 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [10]:
# Convert the model to a TensorFlow Lite model with integer quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable integer quantization (use representative dataset if available)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# If you have a representative dataset, use it here (optional but recommended)
# def representative_dataset_gen():
#     for input_value in dataset.take(100):
#         yield [input_value]

# converter.representative_dataset = representative_dataset_gen

# Convert the model
tflite_quantized_model = converter.convert()

# Save the converted model
with open('model_quantized.tflite', 'wb') as f:
    f.write(tflite_quantized_model)


INFO:tensorflow:Assets written to: /var/folders/k8/3swkzmkd0p5fyj48wzvwgq6h0000gn/T/tmp3xx9o2pr/assets


INFO:tensorflow:Assets written to: /var/folders/k8/3swkzmkd0p5fyj48wzvwgq6h0000gn/T/tmp3xx9o2pr/assets


Saved artifact at '/var/folders/k8/3swkzmkd0p5fyj48wzvwgq6h0000gn/T/tmp3xx9o2pr'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32, name='keras_tensor_478')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  6096600144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6099633168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6099632400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6096600528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6099632208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6099633552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6099634896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6099635088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6099634704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6099633936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6099635472: Tens

W0000 00:00:1728963980.255771 7003993 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1728963980.256793 7003993 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-10-15 11:46:20.257959: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/k8/3swkzmkd0p5fyj48wzvwgq6h0000gn/T/tmp3xx9o2pr
2024-10-15 11:46:20.264618: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-10-15 11:46:20.264635: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/k8/3swkzmkd0p5fyj48wzvwgq6h0000gn/T/tmp3xx9o2pr
2024-10-15 11:46:20.335300: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-10-15 11:46:20.349174: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-10-15 11:46:20.854645: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at 

In [12]:
import os
file_size = os.path.getsize('model_quantized.tflite') / (1024 * 1024)
print(f"Quantized TFLite Model Size: {file_size:.2f} MB")


Quantized TFLite Model Size: 2.39 MB
